### PSF Gaussian Fitting example

In [ ]:
#from toolviper.utils.data import download
#download(file="twhya_selfcal_11chans_lsrk_robust_0.5_niter_0.psf.zip")
#tar xvf twhya_selfcal_11chans_lsrk_robust_0.5_niter_0.psf.zip

Load a CASA psf image

In [2]:
from xradio.image import read_image
xds = read_image(infile='twhya_selfcal_11chans_lsrk_robust_0.5_niter_0.psf', chunks={"l":25, "m":25, "frequency":4} )

[2025-08-28 16:44:31,918]     INFO    viperlog:  J2000 found as system reference frame in CASA image This corresponds to fk5(equinox="j2000") in astropy. Metadata will be written appropriately 
Successful readonly open of default-locked table twhya_selfcal_11chans_lsrk_robust_0.5_niter_0.psf: 1 columns, 1 rows
[2025-08-28 16:44:31,925]     INFO    viperlog:  J2000 found as native reference frame in CASA image This corresponds to fk5(equinox="j2000") in astropy. Metadata will be written appropriately 


In [4]:
xds.BEAM

<xarray.DataArray 'BEAM' (time: 1, frequency: 11, polarization: 1, beam_param: 3)> Size: 264B
dask.array<array, shape=(1, 11, 1, 3), dtype=float64, chunksize=(1, 11, 1, 3), chunktype=numpy.ndarray>
Coordinates:
  * time          (time) float64 8B 5.625e+04
  * frequency     (frequency) float64 88B 3.728e+11 3.728e+11 ... 3.728e+11
  * polarization  (polarization) <U1 4B 'I'
    velocity      (frequency) float64 88B 2.454e+03 1.964e+03 ... -2.454e+03
  * beam_param    (beam_param) <U5 60B 'major' 'minor' 'pa'
Attributes:
    units:    rad

In [3]:

from astroviper.core.image_analysis.psf_gaussian_fit import psf_gaussian_fit
res_xds = psf_gaussian_fit(xds)

is_dask_array: True


/Users/ttsutsum/SWDevel/radps/viper/sandbox/astroviper/src/astroviper/core/image_analysis/psf_gaussian_fit.py:131: NumbaWarning: Code running in object mode won't allow parallel execution despite nogil=True.
  with objmode(res_x="f8[:]"):  # return type anotation


ellipse_params shape: (1, 11, 1, 3)
ellipse_params dtype: float64
ellipse_params content:
 [[[[ 6.67393716e-03  5.42844022e-03 -5.80279179e+01]]

  [[ 6.60730150e-03  5.30830632e-03 -5.61459485e+01]]

  [[ 6.60729463e-03  5.30829988e-03 -5.61459374e+01]]

  [[ 6.60727988e-03  5.30829317e-03 -5.61459891e+01]]

  [[ 6.60793419e-03  5.30740925e-03 -5.61547810e+01]]

  [[ 6.60792559e-03  5.30739828e-03 -5.61549066e+01]]

  [[ 6.60791399e-03  5.30738365e-03 -5.61550033e+01]]

  [[ 6.60790610e-03  5.30736943e-03 -5.61550440e+01]]

  [[ 6.60789787e-03  5.30735946e-03 -5.61550606e+01]]

  [[ 6.60789291e-03  5.30735437e-03 -5.61550838e+01]]

  [[ 6.60788434e-03  5.30734840e-03 -5.61548965e+01]]]]


In [5]:
res_xds.BEAM

<xarray.DataArray 'BEAM' (time: 1, frequency: 11, polarization: 1, beam_param: 3)> Size: 264B
array([[[[ 6.67393716e-03,  5.42844022e-03, -5.80279179e+01]],

        [[ 6.60730150e-03,  5.30830632e-03, -5.61459485e+01]],

        [[ 6.60729463e-03,  5.30829988e-03, -5.61459374e+01]],

        [[ 6.60727988e-03,  5.30829317e-03, -5.61459891e+01]],

        [[ 6.60793419e-03,  5.30740925e-03, -5.61547810e+01]],

        [[ 6.60792559e-03,  5.30739828e-03, -5.61549066e+01]],

        [[ 6.60791399e-03,  5.30738365e-03, -5.61550033e+01]],

        [[ 6.60790610e-03,  5.30736943e-03, -5.61550440e+01]],

        [[ 6.60789787e-03,  5.30735946e-03, -5.61550606e+01]],

        [[ 6.60789291e-03,  5.30735437e-03, -5.61550838e+01]],

        [[ 6.60788434e-03,  5.30734840e-03, -5.61548965e+01]]]])
Coordinates:
  * time          (time) float64 8B 5.625e+04
  * frequency     (frequency) float64 88B 3.728e+11 3.728e+11 ... 3.728e+11
  * polarization  (polarization) <U1 4B 'I'
    velocity      (frequency) float64 88B 2.454e+03 1.964e+03 ... -2.454e+03
  * beam_param    (beam_param) <U5 60B 'major' 'minor' 'pa'
Attributes:
    units:    rad